In [32]:
import sys
from os.path import join as oj
sys.path.insert(1, oj(sys.path[0], '..'))  # insert parent path
sys.path.append('../src')
import numpy as np
import data_pecarn, data_psrc
from sklearn.metrics import confusion_matrix

%matplotlib inline
%load_ext autoreload
%autoreload 2

d = data_pecarn.get_data(use_processed=True, dummy=True, frac_missing_allowed=1, impute_feats=False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


![](../docs/ultrasound.png)

In [33]:
y = d['iai_intervention']
meanings = ['<1%', '1-5%', '6-10%', '11-50%', ">50%", 'Physician did not answer']
y

0        0
1        0
2        0
3        0
4        0
        ..
12039    0
12040    0
12041    0
12042    0
12043    0
Name: iai_intervention, Length: 12044, dtype: int64

In [34]:
fast = d['UltrasoundType'] == 1
abnormal = d['UltrasoundRes'] == 2
fast_abnormal = fast & abnormal

In [38]:
print('# patients', d.shape[0])
print('# iai-i', y.sum())
print('# FAST (UltrasoundType==1)', fast.sum())
print('# abnormal (UltrasoundRes==2)', abnormal.sum())
print('# FAST abnormal', fast_abnormal.sum())

tn, fp, fn, tp = confusion_matrix(y, fast_abnormal).ravel()
print(f'{tp=} {fp=}')
print(f'{fn=} {tp=}')

# patients 12044
# iai-i 203
# FAST (UltrasoundType==1) 848
# abnormal (UltrasoundRes==2) 80
# FAST abnormal 26
tp=12 fp=14
fn=191 tp=12


In [27]:
sorted(list([x for x in d.columns if x.lower().startswith('u')]))

['US4BID',
 'USID',
 'UltrasoundLoc',
 'UltrasoundRes',
 'UltrasoundType',
 'UpperGIAfterLap',
 'UpperGIDay',
 'UpperGIObtain',
 'UpperGIOrgans',
 'UpperGIShow',
 'UpperGIText_Size not documented',
 'UpperGIText_nan',
 'UpperGITm',
 'UrethralInj',
 'Urinalysis',
 'UrinalysisMicro_1496',
 'UrinalysisMicro_1815',
 'UrinalysisMicro_25',
 'UrinalysisMicro_51',
 'UrinalysisMicro_9',
 'UrinalysisMicro_>100',
 'UrinalysisMicro_>50',
 'UrinalysisMicro_nan']